In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

import ipywidgets as widgets
import matplotlib.pyplot as plt

from risk_distributions import EnsembleDistribution
from vivarium_public_health.dataset_manager import Artifact

from vivarium_inputs.interface import get_measure

## We will build the ensemble distribution from artifact data.
You could also pull data using get_measure from vivarium_inputs.interface. 

In [3]:
path = '/share/costeffectiveness/artifacts/obesity/obesity.hdf'
artifact = Artifact(path)

In [4]:
print(artifact)

Artifact containing the following keys:
metadata
	keyspace
	versions
	locations
population
	demographic_dimensions
	structure
	theoretical_minimum_risk_life_expectancy
	age_bins
cause
	all_causes
		cause_specific_mortality
	ischemic_heart_disease
		restrictions
		cause_specific_mortality
		prevalence
		disability_weight
		excess_mortality
		incidence
	ischemic_stroke
		restrictions
		cause_specific_mortality
		prevalence
		disability_weight
		excess_mortality
		incidence
	diabetes_mellitus_type_2
		restrictions
		cause_specific_mortality
		prevalence
		disability_weight
		excess_mortality
		incidence
	asthma
		restrictions
		cause_specific_mortality
		prevalence
		disability_weight
		excess_mortality
		incidence
	gout
		restrictions
		prevalence
		disability_weight
		incidence
	osteoarthritis
		restrictions
		prevalence
		disability_weight
		incidence
	chronic_kidney_disease_due_to_hypertension
		restrictions
		cause_specific_mortality
		prevalence
		disability_weight
		excess_mortalit

In [6]:
# Pull data from the artifact
exp_mean = artifact.load('risk_factor.high_body_mass_index_in_adults.exposure')
exp_std = artifact.load('risk_factor.high_body_mass_index_in_adults.exposure_standard_deviation')
exp_weights = artifact.load('risk_factor.high_body_mass_index_in_adults.exposure_distribution_weights')

# Format the data how EnsembleDistribution expects it
index_columns = ['sex', 'age_group_start', 'age_group_end', 'year_start', 'year_end']

exp_mean = exp_mean.loc[exp_mean.draw == 0]
exp_mean.drop(['draw', 'location', 'parameter'], 'columns', inplace=True)
exp_mean = exp_mean.set_index(index_columns)['value']

exp_std = exp_std.loc[exp_std.draw == 0]
exp_std.drop(['draw', 'location'], 'columns', inplace=True)
exp_std = exp_std.set_index(index_columns)['value']

exp_weights.drop(['location'], 'columns', inplace=True)
exp_weights.set_index(index_columns + ['parameter'], inplace=True)
exp_weights = exp_weights.unstack()
exp_weights.columns = exp_weights.columns.droplevel(0)
exp_weights.columns.name = None
exp_weights.drop('glnorm', 'columns', inplace=True)   # not a distribution we consider for Ensemble
                                                      # you may not have this one.

#### Correct data format

In [7]:
exp_mean.head()

sex     age_group_start  age_group_end  year_start  year_end
Female  0.0              0.019178       1990        1991        0.0
                                        1991        1992        0.0
                                        1992        1993        0.0
                                        1993        1994        0.0
                                        1994        1995        0.0
Name: value, dtype: float64

In [8]:
exp_std.head()

sex     age_group_start  age_group_end  year_start  year_end
Female  0.0              0.019178       1990        1991        0.0
                                        1991        1992        0.0
                                        1992        1993        0.0
                                        1993        1994        0.0
                                        1994        1995        0.0
Name: value, dtype: float64

In [9]:
exp_weights.head()

betasr  exp  gamma  \
sex    age_group_start age_group_end year_start year_end                       
Female 0.0             0.019178      1990       1991         0.0  0.0    0.0   
                                     1991       1992         0.0  0.0    0.0   
                                     1992       1993         0.0  0.0    0.0   
                                     1993       1994         0.0  0.0    0.0   
                                     1994       1995         0.0  0.0    0.0   

                                                          gumbel  invgamma  \
sex    age_group_start age_group_end year_start year_end                     
Female 0.0             0.019178      1990       1991         0.0       0.0   
                                     1991       1992         0.0       0.0   
                                     1992       1993         0.0       0.0   
                                     1993       1994         0.0       0.0   
                                     1994       1995         0.0       0.0   

                                                          invweibull  llogis  \
sex    age_group_start age_group_end year_start year_end                       
Female 0.0             0.019178      1990       1991             0.0     0.0   
                                     1991       1992             0.0     0.0   
                                     1992       1993             0.0     0.0   
                                     1993       1994             0.0     0.0   
                                     1994       1995             0.0     0.0   

                                                          lnorm  mgamma  \
sex    age_group_start age_group_end year_start year_end                  
Female 0.0             0.019178      1990       1991        0.0     0.0   
                                     1991       1992        0.0     0.0   
                                     1992       1993        0.0     0.0   
                                     1993       1994        0.0     0.0   
                                     1994       1995        0.0     0.0   

                                                          mgumbel  norm  \
sex    age_group_start age_group_end year_start year_end                  
Female 0.0             0.019178      1990       1991          0.0   0.0   
                                     1991       1992          0.0   0.0   
                                     1992       1993          0.0   0.0   
                                     1993       1994          0.0   0.0   
                                     1994       1995          0.0   0.0   

                                                          weibull  
sex    age_group_start age_group_end year_start year_end           
Female 0.0             0.019178      1990       1991          0.0  
                                     1991       1992          0.0  
                                     1992       1993          0.0  
                                     1993       1994          0.0  
                                     1994       1995          0.0

In [10]:
# Build an EnsembleDistribution Object
ensemble_dist = EnsembleDistribution(exp_weights, mean=exp_mean, sd=exp_std)

/home/cody/Work/risk_distributions/src/risk_distributions/risk_distributions.py:416: RuntimeWarning: overflow encountered in double_scalars
  return (m - mean_guess) ** 2 + (s ** 2 - var_guess) ** 2
/home/cody/Work/risk_distributions/src/risk_distributions/risk_distributions.py:415: RuntimeWarning: invalid value encountered in double_scalars
  var_guess = scale ** 2 * special.gamma(1 + 2 / shape) - mean_guess ** 2


In [11]:
# Example -- create a bunch of people with BMI 30.0. What's the PDF ?
# Note X has an index of sexes, age groups and years
X = pd.Series(30.0, index=exp_mean.index)
probabilities = ensemble_dist.pdf(X)

In [12]:
# We get NaN where no data is present -- not failure
probabilities.head()

sex     age_group_start  age_group_end  year_start  year_end
Female  0.0              0.019178       1990        1991       NaN
                                        1991        1992       NaN
                                        1992        1993       NaN
                                        1993        1994       NaN
                                        1994        1995       NaN
dtype: float64

In [13]:
probabilities.tail()

sex   age_group_start  age_group_end  year_start  year_end
Male  95.0             125.0          2013        2014        0.057092
                                      2014        2015        0.057176
                                      2015        2016        0.057256
                                      2016        2017        0.057298
                                      2017        2018        0.057267
dtype: float64

In [14]:
# Example -- create a bunch of people with BMI at the median - 50%
# Note q has an index of sexes, age groups and years
q = pd.Series(0.5, index=exp_mean.index)
percentages = ensemble_dist.ppf(q)

In [15]:
percentages.head()

sex     age_group_start  age_group_end  year_start  year_end
Female  0.0              0.019178       1990        1991       NaN
                                        1991        1992       NaN
                                        1992        1993       NaN
                                        1993        1994       NaN
                                        1994        1995       NaN
dtype: float64

In [16]:
percentages.tail()

sex   age_group_start  age_group_end  year_start  year_end
Male  95.0             125.0          2013        2014        31.161007
                                      2014        2015        31.220549
                                      2015        2016        31.265810
                                      2016        2017        31.298981
                                      2017        2018        31.322901
dtype: float64

In [17]:
#  Example plots
# I roughly bounded age and year
@widgets.interact()
def plot_pdf(age_start=range(20,95,5), year_start=range(1996, 2015), sex=['Male', 'Female']):
    index_mask = (sex, age_start, age_start + 5, year_start, year_start + 1)
    sample_mean = exp_mean.loc[index_mask]
    sample_std = exp_std.loc[index_mask]
    sample_weights = exp_weights.loc[index_mask]

    sample_ensemble_dist = EnsembleDistribution(sample_weights, mean=sample_mean, sd=sample_std)

    x = np.linspace(10, 90, num=100)  # BMI from 10 to 90
    q = np.linspace(0.01, 0.99, num=100)  # Percentages from 1 to 99

    fig = plt.figure(figsize=(10, 5))
    plt.plot(sample_ensemble_dist.pdf(x))

    plt.ylim(0.0, 0.15)
    plt.title("PDF Example Plot")
    plt.ylabel("Probability")
    plt.xlabel("BMI")
   
    plt.show()
    
    fig = plt.figure(figsize=(10, 5))
    plt.plot(sample_ensemble_dist.ppf(q))
    
    plt.ylim(0.0, 75.0)
    plt.title("PPF Example plot")
    plt.ylabel('BMI')
    plt.xlabel('Percentile')
    
    plt.show()

interactive(children=(Dropdown(description='age_start', options=(20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 7…